In [1]:
using Plots
using DifferentialEquations
using Interact
using WebIO
using BenchmarkTools
using Distributions
using Statistics
using XLSX

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-6306203360074091235\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-10684824309770734242\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-6306203360074091235\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
sir_ode = @ode_def SIRModel begin
    dS = -β*S*I
    dI = β*S*I-γ*I
    dR = γ*I
    end β γ

(::SIRModel{var"###ParameterizedDiffEqFunction#428",var"###ParameterizedTGradFunction#429",var"###ParameterizedJacobianFunction#430",Nothing,Nothing,ModelingToolkit.ODESystem}) (generic function with 1 method)

In [8]:
@manipulate throttle=0.01 for 
    conf=slider(30:1:200,label="Duration of Quarantine"), 
    c3=slider(5:0.1:15,label="Contact Coefficient"),
    p3=slider(0.005:0.001:0.03,label="Probability of Infection"), 
    Tglob=slider(100:1:500, label="Length")
######PARAMETERS#####

#Global parameters
n=67000000.0 #Population
I0=100.0 #Infected at start
d=15.0 #Infectious period
ν=1/d #Transmission factor
##Tglob=200.0 #Number of days modelled
#conf=45
div=15.0 #Day when measures are implemented
#hmax=0.006 #Maximal Capacity of hospitals

#Initial Scenario
c1=13.4 #Contact coefficient
p1=0.02 #Probability of infection in contact
τ1=c1*p1 #β parameter
#########
Tmax=round(Int64,div) 
tend=convert(Int64, Tmax*10)
t=1:tend
parms = [τ1,ν]
init = [1-I0/n,I0/n,0.0]
tspan = (0.0,Tmax)
########

#Scenario after shock
c2=c1*0.7 #contact coefficient
p2=p1*0.7 #probability of infection in contact
τ2=c2*p2 #β parameter
#########
Tmax2=round(Int64,conf)
parms2 = [τ2,ν] 
tspan2 = (0.0,Tmax2)
tend2=convert(Int64, Tmax2*10)
t2=tend+1:tend2+tend
#########
    
#End of Quarantine Scenario
#c3=c1 #contact coefficient
#p3=p1 #probability of infection in contact
τ3=c3*p3 #β parameter
#########
Tmax3=round(Int64,Tglob-conf-Tmax)
parms3 = [τ3,ν] 
tspan3 = (0.0,Tmax3)
tend3=convert(Int64, Tmax3*10)
t3=tend2+1+tend:tend3+tend2+tend
#########

#Counterfactual
tendc=convert(Int64, Tglob*10)
tc=1:tend4
tspanc = (0.0,Tglob)
##################################

#####Solving######

#Initial ODE system
sir_prob = ODEProblem(sir_ode,init,tspan,parms)
sir_sol = DifferentialEquations.solve(sir_prob,saveat = 0.1)
#Storing results
Inew=fill(0.0,tend)
Snew=fill(0.0,tend)
Rnew=fill(0.0,tend)
for i in 1:tend
    Snew[i]=sir_sol.u[i][1]
    Inew[i]=sir_sol.u[i][2]
    Rnew[i]=sir_sol.u[i][3]
end
#####

#New Initial values
I02=Inew[tend]
R02=Rnew[tend]
S02=Snew[tend]
init2 = [S02,I02,R02]
#After Shock ODE system
sir_prob2 = ODEProblem(sir_ode,init2,tspan2,parms2)
sir_sol2 = DifferentialEquations.solve(sir_prob2,saveat = 0.1)
#Storing results
Inew2=fill(0.0,tend2)
Snew2=fill(0.0,tend2)
Rnew2=fill(0.0,tend2)
for i in 1:tend2
    Snew2[i]=sir_sol2.u[i][1]
    Inew2[i]=sir_sol2.u[i][2]
    Rnew2[i]=sir_sol2.u[i][3]
end
I03=Inew2[tend2]
R03=Rnew2[tend2]
S03=Snew2[tend2]
init3 = [S03,I03,R03]
sir_prob3=ODEProblem(sir_ode,init3,tspan3,parms3)
sir_sol3=DifferentialEquations.solve(sir_prob3,saveat=0.1)
Inew3=fill(0.0,tend3)
Snew3=fill(0.0,tend3)
Rnew3=fill(0.0,tend3)
for i in 1:tend3
    Snew3[i]=sir_sol3.u[i][1]
    Inew3[i]=sir_sol3.u[i][2]
    Rnew3[i]=sir_sol3.u[i][3]
end
    
St=vcat(Snew,Snew2,Snew3)
It=vcat(Inew,Inew2,Inew3)
Rt=vcat(Rnew,Rnew2,Rnew3)
tt=vcat(t./10,t2./10,t3./10)
#####

#Counterfactual ODE System
sir_probc = ODEProblem(sir_ode,init,tspanc,parms)
sir_solc = DifferentialEquations.solve(sir_probc,saveat = 0.1)
#Storing results
Inewc=fill(0.0,tendc)
Snewc=fill(0.0,tendc)
Rnewc=fill(0.0,tendc)
for i in 1:tendc
    Snewc[i]=sir_solc.u[i][1]
    Inewc[i]=sir_solc.u[i][2]
    Rnewc[i]=sir_solc.u[i][3]
end
tc=tc./10
##############

######Plots######

#Quarantine scenario
plot(tt,St*100,label="Susceptible",color=:blue,width=3,grid=false) #Susceptible
plot!(tt,It*100,label="Infected",color=:red,width=3) #Infected
plot!(tt,Rt*100,label="Removed",color=:green,width=3) #Removed
#Counterfactual scenario
plot!(tc,Snew4*100,label="",color=:blue,linestyle=:dash) #Susceptible
plot!(tc,Inew4*100,label="",color=:red,linestyle=:dash) #Infected
plot!(tc,Rnew4*100,label="",color=:green,linestyle=:dash) #Removed
#Reference lines
plot!([findall(x->x==maximum(It),It)[1][1]/10],seriestype="vline",label="Peak of Infections",color=:red, linestyle=:solid, width=2)
plot!([Tmax],seriestype="vline",label="Start of Quarantine",color=:black,linealpha=0.5,width=2) #Implementation date
plot!([conf+Tmax],seriestype="vline",label="End of Quarantine",color=:black,linestyle=:dot,width=2)#End of quarantine date
#plot!([hmax],seriestype="hline",label="",color=:black,linestyle=:dot,width=2) #Hospital Capacity
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Duration of Quarantine"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 171,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
86, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/danell/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/danell/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/danell/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/danell/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/danell/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000117d4c910, Task (runnable) @0x0000000117d4c910), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",\"36\",\"37\",\"38\",\"39\",\"40\",\"41\",\"42\",\"43\",\"44\",\"45\",\"46\",\"47\",\"48\",\"49\",\"50\",\"51\",\"52\",\"53\",\"54\",\"55\",\"56\",\"57\",\"58\",\"59\",\"60\",\"61\"

In [39]:
#@manipulate throttle=0.01 for 
 #   tshock=slider(1:1:Tglob,label="Time of Individual Shock"),
  #  nshock=slider(0.1:0.1:10.0,label="People Infected by Shock")
    #conf=slider(30:1:200,label="Duration of Quarantine"), 
    #c3=slider(5:0.1:15,label="Contact Coefficient"),
    #p3=slider(0.005:0.001:0.03,label="Probability of Infection"), 
   # Tglob=slider(100:1:500, label="Length")
######PARAMETERS#####

#Global parameters
n=67000000.0 #Population
I0=100.0 #Infected at start
d=15.0 #Infectious period
ν=1/d #Transmission factor
Tglob=200.0 #Number of days modelled
conf=45
div=15.0 #Day when measures are implemented
#hmax=0.006 #Maximal Capacity of hospitals

#Initial Scenario
c1=13.4 #Contact coefficient
p1=0.02 #Probability of infection in contact
τ1=c1*p1 #β parameter
#########
Tmax=round(Int64,div) 
tend=convert(Int64, Tmax*100)
t=1:tend
parms = [τ1,ν]
init = [1-I0/n,I0/n,0.0]
tspan = (0.0,Tmax)
########

#Scenario after shock
c2=c1*0.7 #contact coefficient
p2=p1*0.7 #probability of infection in contact
τ2=c2*p2 #β parameter
#########
Tmax2=round(Int64,conf)
parms2 = [τ2,ν] 
tspan2 = (0.0,Tmax2)
tend2=convert(Int64, Tmax2*100)
t2=tend+1:tend2+tend
#########
    
#End of Quarantine Scenario
c3=c1 #contact coefficient
p3=p1 #probability of infection in contact
τ3=c3*p3 #β parameter
#########
Tmax3=round(Int64,Tglob-conf-Tmax)
parms3 = [τ3,ν] 
tspan3 = (0.0,Tmax3)
tend3=convert(Int64, Tmax3*100)
t3=tend2+1+tend:tend3+tend2+tend
    
        
#Individual Shock
nshock=200.0
tshock=1

if tshock<Tmax
    nshock=nshock*p1
    parms4=parms
    init4=init
    tspan4=(0.0,tshock)
    tend4=convert(Int64,tshock*100)
    t4=1:tend4
    sir_prob4 = ODEProblem(sir_ode,init4,tspan4,parms4)
    sir_sol4 = DifferentialEquations.solve(sir_prob4,saveat = 0.01)
Inew=fill(0.0,tend4)
Snew=fill(0.0,tend4)
Rnew=fill(0.0,tend4)
for i in 1:tend4
    Snew[i]=sir_sol4.u[i][1]
    Inew[i]=sir_sol4.u[i][2]
    Rnew[i]=sir_sol4.u[i][3]
end
I02=Inew[tend4]
R02=Rnew[tend4]
S02=Snew[tend4]
        parms5=parms4
        init5=[S02,I02,R02]
        tspan5=(0.0,Tmax-tshock)
        tend5=convert(Int64,(Tmax-tshock)*100)
        t5=tend4+1:tend5+tend4
        sir_prob5=ODEProblem(sir_ode,init5,tspan5,parms5)
        sir_sol5=DifferentialEquations.solve(sir_prob5,saveat = 0.01)
        Inew2=fill(0.0,tend5)
        Snew2=fill(0.0,tend5)
        Rnew2=fill(0.0,tend5)
        for i in 1:tend5
    Snew2[i]=sir_sol5.u[i][1]
    Inew2[i]=sir_sol5.u[i][2]
    Rnew2[i]=sir_sol5.u[i][3]
end
I03=Inew2[tend5]
R03=Rnew2[tend5]
S03=Snew2[tend5]
        parms6=parms2
        init6=[S03,I03,R03]
        tspan6=(0.0,Tmax2)
        tend6=convert(Int64,Tmax2*100)
        t6=tend4+1+tend5:tend5+tend4+tend6
        sir_prob6=ODEProblem(sir_ode,init6,tspan6,parms6)
        sir_sol6=DifferentialEquations.solve(sir_prob6,saveat = 0.01)
        Inew3=fill(0.0,tend6)
        Snew3=fill(0.0,tend6)
        Rnew3=fill(0.0,tend6)
        for i in 1:tend6
    Snew3[i]=sir_sol6.u[i][1]
    Inew3[i]=sir_sol6.u[i][2]
    Rnew3[i]=sir_sol6.u[i][3]
        end
I04=Inew3[tend6]
R04=Rnew3[tend6]
S04=Snew3[tend6]
        parms7=parms3
        init7=[S04,I04,R04]
        tspan7=(0.0,Tmax3)
        tend7=convert(Int64,Tmax3*100)
        t7=tend4+tend5+tend6+1:tend4+tend5+tend6+tend7
        sir_prob7=ODEProblem(sir_ode,init7,tspan7,parms7)
        sir_sol7=DifferentialEquations.solve(sir_prob7,saveat=0.01)
        Inew4=fill(0.0,tend7)
        Snew4=fill(0.0,tend7)
        Rnew4=fill(0.0,tend7)
        for i in 1:tend7
            Snew4[i]=sir_sol7.u[i][1]
            Inew4[i]=sir_sol7.u[i][2]
            Rnew4[i]=sir_sol7.u[i][3]
        end
St=vcat(Snew,Snew2,Snew3,Snew4)
It=vcat(Inew,Inew2,Inew3,Inew4)
Rt=vcat(Rnew,Rnew2,Rnew3,Rnew4)
        elseif tshock >=Tmax && tshock<Tmax2+Tmax
    nshock=nshock*p2
   parms4=parms
    init4=init
    tspan4=(0.0,Tmax)
    tend4=convert(Int64,Tmax*100)
    t4=1:tend4
    sir_prob4 = ODEProblem(sir_ode,init4,tspan4,parms4)
    sir_sol4 = DifferentialEquations.solve(sir_prob4,saveat = 0.01)
Inew=fill(0.0,tend4)
Snew=fill(0.0,tend4)
Rnew=fill(0.0,tend4)
for i in 1:tend4
    Snew[i]=sir_sol4.u[i][1]
    Inew[i]=sir_sol4.u[i][2]
    Rnew[i]=sir_sol4.u[i][3]
end
I02=Inew[tend4]
R02=Rnew[tend4]
S02=Snew[tend4]
        parms5=parms4
        init5=[S02,I02,R02]
        tspan5=(0.0,tshock-Tmax)
        tend5=convert(Int64,(tshock-Tmax)*100)
        t5=tend4+1:tend5+tend4
        sir_prob5=ODEProblem(sir_ode,init5,tspan5,parms5)
        sir_sol5=DifferentialEquations.solve(sir_prob5,saveat = 0.01)
        Inew2=fill(0.0,tend5)
        Snew2=fill(0.0,tend5)
        Rnew2=fill(0.0,tend5)
        for i in 1:tend5
    Snew2[i]=sir_sol5.u[i][1]
    Inew2[i]=sir_sol5.u[i][2]
    Rnew2[i]=sir_sol5.u[i][3]
end
I03=Inew2[tend5]
R03=Rnew2[tend5]
S03=Snew2[tend5]
        parms6=parms2
        init6=[S03,I03,R03]
        tspan6=(0.0,Tmax2-(tshock-Tmax))
        tend6=convert(Int64,(Tmax2-(tshock-Tmax))*100)
        t6=tend4+1+tend5:tend5+tend4+tend6
        sir_prob6=ODEProblem(sir_ode,init6,tspan6,parms6)
        sir_sol6=DifferentialEquations.solve(sir_prob6,saveat = 0.01)
        Inew3=fill(0.0,tend6)
        Snew3=fill(0.0,tend6)
        Rnew3=fill(0.0,tend6)
        for i in 1:tend6
    Snew3[i]=sir_sol6.u[i][1]
    Inew3[i]=sir_sol6.u[i][2]
    Rnew3[i]=sir_sol6.u[i][3]
        end
I04=Inew3[tend6]
R04=Rnew3[tend6]
S04=Snew3[tend6]
        parms7=parms3
        init7=[S04,I04,R04]
        tspan7=(0.0,Tmax3)
        tend7=convert(Int64,Tmax3*100)
        t7=tend4+tend5+tend6+1:tend4+tend5+tend6+tend7
        sir_prob7=ODEProblem(sir_ode,init7,tspan7,parms7)
        sir_sol7=DifferentialEquations.solve(sir_prob7,saveat=0.01)
        Inew4=fill(0.0,tend7)
        Snew4=fill(0.0,tend7)
        Rnew4=fill(0.0,tend7)
        for i in 1:tend7
            Snew4[i]=sir_sol7.u[i][1]
            Inew4[i]=sir_sol7.u[i][2]
            Rnew4[i]=sir_sol7.u[i][3]
        end
St=vcat(Snew,Snew2,Snew3,Snew4)
It=vcat(Inew,Inew2,Inew3,Inew4)
Rt=vcat(Rnew,Rnew2,Rnew3,Rnew4)
    elseif tshock >=Tmax2+Tmax     
    nshock=nshock*p3
     parms4=parms
    init4=init
    tspan4=(0.0,Tmax)
    tend4=convert(Int64,Tmax*100)
    t4=1:tend4
    sir_prob4 = ODEProblem(sir_ode,init4,tspan4,parms4)
    sir_sol4 = DifferentialEquations.solve(sir_prob4,saveat = 0.01)
Inew=fill(0.0,tend4)
Snew=fill(0.0,tend4)
Rnew=fill(0.0,tend4)
for i in 1:tend4
    Snew[i]=sir_sol4.u[i][1]
    Inew[i]=sir_sol4.u[i][2]
    Rnew[i]=sir_sol4.u[i][3]
end
I02=Inew[tend4]
R02=Rnew[tend4]
S02=Snew[tend4]
        parms5=parms4
        init5=[S02,I02,R02]
        tspan5=(0.0,Tmax2)
        tend5=convert(Int64,Tmax2*100)
        t5=tend4+1:tend5+tend4
        sir_prob5=ODEProblem(sir_ode,init5,tspan5,parms5)
        sir_sol5=DifferentialEquations.solve(sir_prob5,saveat = 0.01)
        Inew2=fill(0.0,tend5)
        Snew2=fill(0.0,tend5)
        Rnew2=fill(0.0,tend5)
        for i in 1:tend5
    Snew2[i]=sir_sol5.u[i][1]
    Inew2[i]=sir_sol5.u[i][2]
    Rnew2[i]=sir_sol5.u[i][3]
end
I03=Inew2[tend5]
R03=Rnew2[tend5]
S03=Snew2[tend5]
        parms6=parms2
        init6=[S03,I03,R03]
        tspan6=(0.0,tshock-Tmax-Tmax2)
        tend6=convert(Int64,(tshock-Tmax-Tmax2)*100)
        t6=tend4+1+tend5:tend5+tend4+tend6
        sir_prob6=ODEProblem(sir_ode,init6,tspan6,parms6)
        sir_sol6=DifferentialEquations.solve(sir_prob6,saveat = 0.01)
        Inew3=fill(0.0,tend6)
        Snew3=fill(0.0,tend6)
        Rnew3=fill(0.0,tend6)
        for i in 1:tend6
    Snew3[i]=sir_sol6.u[i][1]
    Inew3[i]=sir_sol6.u[i][2]
    Rnew3[i]=sir_sol6.u[i][3]
        end
I04=Inew3[tend6]+nshock/n
R04=Rnew3[tend6]
S04=Snew3[tend6]-nshock/n
        parms7=parms3
        init7=[S04,I04,R04]
        tspan7=(0.0,Tmax3-(tshock-Tmax-Tmax2))
        tend7=convert(Int64,(Tmax3-(tshock-Tmax-Tmax2))*100)
        t7=tend4+tend5+tend6+1:tend4+tend5+tend6+tend7
        sir_prob7=ODEProblem(sir_ode,init7,tspan7,parms7)
        sir_sol7=DifferentialEquations.solve(sir_prob7,saveat=0.01)
        Inew4=fill(0.0,tend7)
        Snew4=fill(0.0,tend7)
        Rnew4=fill(0.0,tend7)
        for i in 1:tend7
            Snew4[i]=sir_sol7.u[i][1]
            Inew4[i]=sir_sol7.u[i][2]
            Rnew4[i]=sir_sol7.u[i][3]
        end
St=vcat(Snew,Snew2,Snew3,Snew4)
It=vcat(Inew,Inew2,Inew3,Inew4)
Rt=vcat(Rnew,Rnew2,Rnew3,Rnew4)
    end


if tshock<Tmax
    nshock=nshock*p1
    parms4=parms
    init4=init
    tspan4=(0.0,tshock)
    tend4=convert(Int64,tshock*100)
    t4=1:tend4
    sir_prob4 = ODEProblem(sir_ode,init4,tspan4,parms4)
    sir_sol4 = DifferentialEquations.solve(sir_prob4,saveat = 0.01)
Inew1=fill(0.0,tend4)
Snew1=fill(0.0,tend4)
Rnew1=fill(0.0,tend4)
for i in 1:tend4
    Snew1[i]=sir_sol4.u[i][1]
    Inew1[i]=sir_sol4.u[i][2]
    Rnew1[i]=sir_sol4.u[i][3]
end
I2=Inew1[tend4]+nshock/n
R2=Rnew1[tend4]
S2=Snew1[tend4]-nshock/n
        parms5=parms4
        init5=[S2,I2,R2]
        tspan5=(0.0,Tmax-tshock)
        tend5=convert(Int64,(Tmax-tshock)*100)
        t5=tend4+1:tend5+tend4
        sir_prob5=ODEProblem(sir_ode,init5,tspan5,parms5)
        sir_sol5=DifferentialEquations.solve(sir_prob5,saveat = 0.01)
        Inew5=fill(0.0,tend5)
        Snew5=fill(0.0,tend5)
        Rnew5=fill(0.0,tend5)
        for i in 1:tend5
    Snew5[i]=sir_sol5.u[i][1]
    Inew5[i]=sir_sol5.u[i][2]
    Rnew5[i]=sir_sol5.u[i][3]
end
I3=Inew5[tend5]
R3=Rnew5[tend5]
S3=Snew5[tend5]
        parms6=parms2
        init6=[S3,I3,R3]
        tspan6=(0.0,Tmax2)
        tend6=convert(Int64,Tmax2*100)
        t6=tend4+1+tend5:tend5+tend4+tend6
        sir_prob6=ODEProblem(sir_ode,init6,tspan6,parms6)
        sir_sol6=DifferentialEquations.solve(sir_prob6,saveat = 0.01)
        Inew6=fill(0.0,tend6)
        Snew6=fill(0.0,tend6)
        Rnew6=fill(0.0,tend6)
        for i in 1:tend6
    Snew6[i]=sir_sol6.u[i][1]
    Inew6[i]=sir_sol6.u[i][2]
    Rnew6[i]=sir_sol6.u[i][3]
        end
I4=Inew6[tend6]
R4=Rnew6[tend6]
S4=Snew6[tend6]
        parms7=parms3
        init7=[S4,I4,R4]
        tspan7=(0.0,Tmax3)
        tend7=convert(Int64,Tmax3*100)
        t7=tend4+tend5+tend6+1:tend4+tend5+tend6+tend7
        sir_prob7=ODEProblem(sir_ode,init7,tspan7,parms7)
        sir_sol7=DifferentialEquations.solve(sir_prob7,saveat=0.01)
        Inew7=fill(0.0,tend7)
        Snew7=fill(0.0,tend7)
        Rnew7=fill(0.0,tend7)
        for i in 1:tend7
            Snew7[i]=sir_sol7.u[i][1]
            Inew7[i]=sir_sol7.u[i][2]
            Rnew7[i]=sir_sol7.u[i][3]
        end
Stshock=vcat(Snew1,Snew5,Snew6,Snew7)
Itshock=vcat(Inew1,Inew5,Inew6,Inew7)
Rtshock=vcat(Rnew1,Rnew5,Rnew6,Rnew7)
        elseif tshock >=Tmax && tshock<Tmax2+Tmax
    nshock=nshock*p2
   parms4=parms
    init4=init
    tspan4=(0.0,Tmax)
    tend4=convert(Int64,Tmax*100)
    t4=1:tend4
    sir_prob4 = ODEProblem(sir_ode,init4,tspan4,parms4)
    sir_sol4 = DifferentialEquations.solve(sir_prob4,saveat = 0.01)
Inew1=fill(0.0,tend4)
Snew1=fill(0.0,tend4)
Rnew1=fill(0.0,tend4)
for i in 1:tend4
    Snew1[i]=sir_sol4.u[i][1]
    Inew1[i]=sir_sol4.u[i][2]
    Rnew1[i]=sir_sol4.u[i][3]
end
I2=Inew1[tend4]
R2=Rnew1[tend4]
S2=Snew1[tend4]
        parms5=parms4
        init5=[S2,I2,R2]
        tspan5=(0.0,tshock-Tmax)
        tend5=convert(Int64,(tshock-Tmax)*100)
        t5=tend4+1:tend5+tend4
        sir_prob5=ODEProblem(sir_ode,init5,tspan5,parms5)
        sir_sol5=DifferentialEquations.solve(sir_prob5,saveat = 0.01)
        Inew5=fill(0.0,tend5)
        Snew5=fill(0.0,tend5)
        Rnew5=fill(0.0,tend5)
        for i in 1:tend5
    Snew5[i]=sir_sol5.u[i][1]
    Inew5[i]=sir_sol5.u[i][2]
    Rnew5[i]=sir_sol5.u[i][3]
end
I3=Inew5[tend5]+nshock/n
R3=Rnew5[tend5]
S3=Snew5[tend5]-nshock/n
        parms6=parms2
        init6=[S3,I3,R3]
        tspan6=(0.0,Tmax2-(tshock-Tmax))
        tend6=convert(Int64,(Tmax2-(tshock-Tmax))*100)
        t6=tend4+1+tend5:tend5+tend4+tend6
        sir_prob6=ODEProblem(sir_ode,init6,tspan6,parms6)
        sir_sol6=DifferentialEquations.solve(sir_prob6,saveat = 0.01)
        Inew6=fill(0.0,tend6)
        Snew6=fill(0.0,tend6)
        Rnew6=fill(0.0,tend6)
        for i in 1:tend6
    Snew6[i]=sir_sol6.u[i][1]
    Inew6[i]=sir_sol6.u[i][2]
    Rnew6[i]=sir_sol6.u[i][3]
        end
I4=Inew6[tend6]
R4=Rnew6[tend6]
S4=Snew6[tend6]
        parms7=parms3
        init7=[S4,I4,R4]
        tspan7=(0.0,Tmax3)
        tend7=convert(Int64,Tmax3*100)
        t7=tend4+tend5+tend6+1:tend4+tend5+tend6+tend7
        sir_prob7=ODEProblem(sir_ode,init7,tspan7,parms7)
        sir_sol7=DifferentialEquations.solve(sir_prob7,saveat=0.01)
        Inew7=fill(0.0,tend7)
        Snew7=fill(0.0,tend7)
        Rnew7=fill(0.0,tend7)
        for i in 1:tend7
            Snew7[i]=sir_sol7.u[i][1]
            Inew7[i]=sir_sol7.u[i][2]
            Rnew7[i]=sir_sol7.u[i][3]
        end
Stshock=vcat(Snew1,Snew5,Snew6,Snew7)
Itshock=vcat(Inew1,Inew5,Inew6,Inew7)
Rtshock=vcat(Rnew1,Rnew5,Rnew6,Rnew7)
    else       
    nshock=nshock*p3
     parms4=parms
    init4=init
    tspan4=(0.0,Tmax)
    tend4=convert(Int64,Tmax*100)
    t4=1:tend4
    sir_prob4 = ODEProblem(sir_ode,init4,tspan4,parms4)
    sir_sol4 = DifferentialEquations.solve(sir_prob4,saveat = 0.01)
Inew1=fill(0.0,tend4)
Snew1=fill(0.0,tend4)
Rnew1=fill(0.0,tend4)
for i in 1:tend4
    Snew1[i]=sir_sol4.u[i][1]
    Inew1[i]=sir_sol4.u[i][2]
    Rnew1[i]=sir_sol4.u[i][3]
end
I2=Inew1[tend4]
R2=Rnew1[tend4]
S2=Snew1[tend4]
        parms5=parms4
        init5=[S2,I2,R2]
        tspan5=(0.0,Tmax2)
        tend5=convert(Int64,Tmax2*100)
        t5=tend4+1:tend5+tend4
        sir_prob5=ODEProblem(sir_ode,init5,tspan5,parms5)
        sir_sol5=DifferentialEquations.solve(sir_prob5,saveat = 0.01)
        Inew5=fill(0.0,tend5)
        Snew5=fill(0.0,tend5)
        Rnew5=fill(0.0,tend5)
        for i in 1:tend5
    Snew5[i]=sir_sol5.u[i][1]
    Inew5[i]=sir_sol5.u[i][2]
    Rnew5[i]=sir_sol5.u[i][3]
end
I3=Inew5[tend5]
R3=Rnew5[tend5]
S3=Snew5[tend5]
        parms6=parms2
        init6=[S3,I3,R3]
        tspan6=(0.0,tshock-Tmax-Tmax2)
        tend6=convert(Int64,(tshock-Tmax-Tmax2)*100)
        t6=tend4+1+tend5:tend5+tend4+tend6
        sir_prob6=ODEProblem(sir_ode,init6,tspan6,parms6)
        sir_sol6=DifferentialEquations.solve(sir_prob6,saveat = 0.01)
        Inew6=fill(0.0,tend6)
        Snew6=fill(0.0,tend6)
        Rnew6=fill(0.0,tend6)
        for i in 1:tend6
    Snew6[i]=sir_sol6.u[i][1]
    Inew6[i]=sir_sol6.u[i][2]
    Rnew6[i]=sir_sol6.u[i][3]
        end
I4=Inew6[tend6]+nshock/n
R4=Rnew6[tend6]
S4=Snew6[tend6]-nshock/n
        parms7=parms3
        init7=[S4,I4,R4]
        tspan7=(0.0,Tmax3-(tshock-Tmax-Tmax2))
        tend7=convert(Int64,(Tmax3-(tshock-Tmax-Tmax2))*100)
        t7=tend4+tend5+tend6+1:tend4+tend5+tend6+tend7
        sir_prob7=ODEProblem(sir_ode,init7,tspan7,parms7)
        sir_sol7=DifferentialEquations.solve(sir_prob7,saveat=0.01)
        Inew7=fill(0.0,tend7)
        Snew7=fill(0.0,tend7)
        Rnew7=fill(0.0,tend7)
        for i in 1:tend7
            Snew7[i]=sir_sol7.u[i][1]
            Inew7[i]=sir_sol7.u[i][2]
            Rnew7[i]=sir_sol7.u[i][3]
        end
Stshock=vcat(Snew1,Snew5,Snew6,Snew7)
Itshock=vcat(Inew1,Inew5,Inew6,Inew7)
Rtshock=vcat(Rnew1,Rnew5,Rnew6,Rnew7)
    end
        
diffvec=Itshock.-It
result=diffvec[findall(x->x==maximum(diffvec),diffvec)]
result=round(Int64,result[1]*n)
if result==1 
    println("Your jogging infected $result person.")
else
    println("Your jogging infected $result people.")
end

Your jogging infected 5664 people.


In [86]:
function MCMC(data::Array{Real,2},var::Float64,n::Int64)
    β=fill(0.0,n)
    β[1]=0.25
    y=data[:,2]
    for i in 2:n
        βc=β[i-1]
        βnew=rand(Normal(βc,var))
        outputc=(SIR([data[1,1],data[1,2],data[1,3]],βc,length(y)))
        Itc=outputc[2]
        outputnew=(SIR([data[1,1],data[1,2],data[1,3]],βnew,length(y)))
        Itnew=outputnew[2]
        b=fill(0.0,length(y))
        for j in 1:length(y)
            b[j]=(y[j]-Itc[j])^2-(y[j]-Itnew[j])^2
        end
        loga=1/(2*var)*sum(b)
        α=exp(loga)
        unif=rand(Uniform(0,1))
        if α<unif
            β[i]=βc
        else
            β[i]=βnew
        end
    end
    return β
end

MCMC (generic function with 3 methods)

In [87]:
function SIR(init::Array{Float64},β::Float64,Tglob::Int64)
#Global parameters
d=15.0 #Infectious period
ν=1/d #Transmission factor

τ1=β
#########
Tmax=round(Int64,Tglob) 
tend=convert(Int64, Tmax*100)
t=1:tend
parms = [τ1,ν]
tspan = (0.0,Tmax)
########

sir_prob = ODEProblem(sir_ode,init,tspan,parms)
sir_sol = DifferentialEquations.solve(sir_prob,saveat = 0.01)
#Storing results
Inew=fill(0.0,tend)
Snew=fill(0.0,tend)
Rnew=fill(0.0,tend)
for i in 1:tend
    Snew[i]=sir_sol.u[i][1]
    Inew[i]=sir_sol.u[i][2]
    Rnew[i]=sir_sol.u[i][3]
end
#####

St=fill(0.0,Tmax)
for i in 1:Tmax
    St[i]=Snew[i*100]
end
    Rt=fill(0.0,Tmax)
for i in 1:Tmax
    Rt[i]=Rnew[i*100]
end
    It=fill(0.0,Tmax)
for i in 1:Tmax
    It[i]=Inew[i*100]
end
    return St,It,Rt
end

SIR (generic function with 3 methods)

In [93]:
x=XLSX.readxlsx("SIR.xlsx")["Feuille 1"][:]
param=[x[:,1] x[:,2] x[:,3]]
MCMC(param,0.01,1000)

1000-element Array{Float64,1}:
 0.25                
 0.24808626517234875 
 0.2528157129517311  
 0.254580658320024   
 0.27088943192441883 
 0.27291366557777536 
 0.28090608189478794 
 0.2975914267421407  
 0.2951298050633731  
 0.30669342665459653 
 0.308799930008604   
 0.30693211906424067 
 0.32443772635101104 
 ⋮                   
 0.047087738744434095
 0.04453460047694567 
 0.06207526126099949 
 0.059116492796113015
 0.05835387743061593 
 0.05334121430322936 
 0.057573921949002474
 0.045910233969215314
 0.05073708757366132 
 0.04408400136599524 
 0.033200601287625954
 0.030238621625428876